In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from six.moves import cPickle 
import time

In [2]:
def load_cifar10(dataset_number):

    for i in range(dataset_number):
        filename = '/home/alvin/notebook_home/ai_lab_image/corpus/cifar-10-batches-py/data_batch_' +str(i+1)
        f = open(filename, 'rb')
        datadict = cPickle.load(f,encoding='latin1')
        f.close()
    
        DX= datadict["data"] # 1000 x (32x32x3)
        DY= datadict['labels']
        #X = DX.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
        X[i*10000:(i+1)*10000] = np.array(DX)
        Y[i*10000:(i+1)*10000] = np.array(DY)
        #break
    return X,Y

In [3]:
cifar_label = { '0':'airplane',
                '1':'automobile',
                '2':'bird',
                '3':'cat',
                '4':'deer',
                '5':'dog',
                '6':'frog',
                '7':'horse',
                '8':'ship',
                '9':'truck'}

In [4]:
dataset_number = 2
X = np.zeros(shape=(dataset_number*10000,32*32*3),dtype='float64')
Y = np.empty([dataset_number * 10000], dtype='uint8')
X,Y = load_cifar10(dataset_number)

In [ ]:
Y[10000:10010]

In [ ]:
# display basic information 
print('dataset count = ' + str(len(X)))
for i in range(10):
    print('number of class '+ str(i) + ': ' + str(Y[np.where(Y==i)].size))

In [ ]:
# display cifar image
#Visualizing CIFAR 10 method 2
X3 = X.reshape(10000*dataset_number, 3, 32, 32).transpose(0,2,3,1)
pic_dim = 6
fig, axes1 = plt.subplots(pic_dim,pic_dim,figsize=(6,6))
for i in range(pic_dim*pic_dim):
    #print('i/pic_dim: ' + str(i/pic_dim) + 'i%pic_dim: ' + str(i%pic_dim))
    axes1[int(i/pic_dim)][int(i%pic_dim)].set_axis_off()
    axes1[int(i/pic_dim)][int(i%pic_dim)].imshow(X3[i])
    axes1[int(i/pic_dim)][int(i%pic_dim)].annotate(cifar_label[str(Y[i])],xy=(0.2,0.6))

In [ ]:
# display cifar image with gray
#Visualizing CIFAR 10 method 2
XG = X.reshape(10000*dataset_number, 3, 1024).transpose(0,2,1).astype("float64") #1000 x 3 x 1024 --> 1000 x 1024 x 3
XG = np.mean(XG,axis=2)                                         #1000 x 1024
XG = XG.reshape(10000*dataset_number, 32, 32)
pic_dim = 6
fig, axes1 = plt.subplots(pic_dim,pic_dim,figsize=(6,6))
for i in range(pic_dim*pic_dim):
    #print('i/pic_dim: ' + str(i/pic_dim) + 'i%pic_dim: ' + str(i%pic_dim))
    axes1[int(i/pic_dim)][int(i%pic_dim)].set_axis_off()
    axes1[int(i/pic_dim)][int(i%pic_dim)].imshow(XG[i], cmap='gray')
    axes1[int(i/pic_dim)][int(i%pic_dim)].annotate(cifar_label[str(Y[i])],xy=(0.2,0.6))

In [ ]:
###############################################################
#   knn main code
###############################################################

In [5]:
XG = X.reshape(10000*dataset_number, 3, 1024).transpose(0,2,1).astype("float64") #10000 x 3 x 1024 --> 10000 x 1024 x 3
XG = np.mean(XG,axis=2)                                         #10000 x 1024

# training set and testing test
t_rate   = 0.8
train_set_num = int(len(X)*t_rate)
train_set   = XG[:train_set_num]
train_label = Y[:train_set_num]
test_set    = XG[train_set_num+1:]
test_label  = Y[train_set_num+1:]

In [6]:
def l1_dist(input_vec, train_set):
    l1_formula = lambda x: sum(abs(input_vec - x))
    l1_dist_vec= l1_formula(train_set)
    return l1_dist_vec

def l2_dist(input_vec, train_set):
    l2_formula = lambda x: np.sqrt(sum((input_vec - x) ** 2))
    l2_dist_vec= l2_formula(train_set)
    return l2_dist_vec

In [7]:
Stime = time.time()
k = 3
correct_num = 0
pred = np.zeros(shape=(len(test_set),1),dtype='uint8')
for i in range(len(test_set)):
    test_sample = test_set[i]
    dict_vec    = l1_dist(test_sample, train_set)
    idx         = np.argpartition(dict_vec,k)[0:k]
    vote        = train_label[idx[0:k]]
    #tmp = np.bincount(vote)
    #print(str(np.bincount(vote)))
    #print(str(np.argmax(tmp)))
    pred[i]     = np.argmax(np.bincount(vote))
    #if i > 100: break
    print(str(test_label[i]) + ' , ' + str(vote) + ' --> ' + str(pred[i]) + '\t progress ' + str(i) + '/' + str(len(test_set)))
    
    if test_label[i] == train_label[idx[0]]:
        correct_num += 1
Etime = time.time()
accuracy = correct_num/len(test_set)
print('knn accuarcy is ' + str(float("{0:.3f}".format(accuracy*100)))+'%')
print('exectution time knn w/ k = ' +str(k)+ ' is ' + str(float(("{0:.2f}".format(Etime-Stime)))) + 's')

9 , [5 2 2] --> [2]	 progress 0/3999
0 , [5 6 7] --> [5]	 progress 1/3999
6 , [4 2 4] --> [4]	 progress 2/3999
0 , [9 9 4] --> [9]	 progress 3/3999
7 , [3 8 6] --> [3]	 progress 4/3999
1 , [4 4 9] --> [4]	 progress 5/3999
4 , [4 3 1] --> [1]	 progress 6/3999
6 , [5 8 3] --> [3]	 progress 7/3999
5 , [0 4 3] --> [0]	 progress 8/3999
0 , [4 4 1] --> [4]	 progress 9/3999
0 , [1 2 6] --> [1]	 progress 10/3999
1 , [7 0 1] --> [0]	 progress 11/3999
3 , [8 6 3] --> [3]	 progress 12/3999
3 , [8 1 5] --> [1]	 progress 13/3999
0 , [2 7 8] --> [2]	 progress 14/3999
9 , [6 3 7] --> [3]	 progress 15/3999
1 , [1 9 1] --> [1]	 progress 16/3999
3 , [1 1 9] --> [1]	 progress 17/3999
1 , [2 0 6] --> [0]	 progress 18/3999
4 , [3 4 6] --> [3]	 progress 19/3999
7 , [2 2 6] --> [2]	 progress 20/3999
7 , [7 6 8] --> [6]	 progress 21/3999
1 , [0 2 3] --> [0]	 progress 22/3999
4 , [1 9 3] --> [1]	 progress 23/3999
7 , [4 7 2] --> [2]	 progress 24/3999
7 , [7 8 0] --> [0]	 progress 25/3999
8 , [4 6 2] --> [2]	 p

2 , [5 9 1] --> [1]	 progress 214/3999
4 , [8 6 0] --> [0]	 progress 215/3999
8 , [1 4 6] --> [1]	 progress 216/3999
6 , [8 6 1] --> [1]	 progress 217/3999
0 , [9 7 7] --> [7]	 progress 218/3999
4 , [7 8 1] --> [1]	 progress 219/3999
2 , [7 3 2] --> [2]	 progress 220/3999
4 , [1 0 5] --> [0]	 progress 221/3999
1 , [4 3 2] --> [2]	 progress 222/3999
7 , [4 9 3] --> [3]	 progress 223/3999
0 , [7 6 8] --> [6]	 progress 224/3999
9 , [8 7 3] --> [3]	 progress 225/3999
7 , [4 2 3] --> [2]	 progress 226/3999
3 , [1 5 2] --> [1]	 progress 227/3999
7 , [7 7 1] --> [7]	 progress 228/3999
0 , [7 5 1] --> [1]	 progress 229/3999
3 , [8 8 6] --> [8]	 progress 230/3999
2 , [4 8 7] --> [4]	 progress 231/3999
1 , [4 0 4] --> [4]	 progress 232/3999
9 , [8 4 1] --> [1]	 progress 233/3999
0 , [7 4 0] --> [0]	 progress 234/3999
5 , [2 1 7] --> [1]	 progress 235/3999
9 , [4 2 7] --> [2]	 progress 236/3999
0 , [1 4 6] --> [1]	 progress 237/3999
8 , [7 6 5] --> [5]	 progress 238/3999
4 , [9 4 1] --> [1]	 prog

4 , [4 9 4] --> [4]	 progress 425/3999
3 , [4 2 2] --> [2]	 progress 426/3999
1 , [8 5 6] --> [5]	 progress 427/3999
6 , [9 9 9] --> [9]	 progress 428/3999
8 , [6 5 0] --> [0]	 progress 429/3999
5 , [4 1 2] --> [1]	 progress 430/3999
3 , [7 0 7] --> [7]	 progress 431/3999
4 , [8 4 0] --> [0]	 progress 432/3999
4 , [2 9 0] --> [0]	 progress 433/3999
8 , [0 0 7] --> [0]	 progress 434/3999
5 , [8 6 8] --> [8]	 progress 435/3999
1 , [9 0 2] --> [0]	 progress 436/3999
3 , [4 5 7] --> [4]	 progress 437/3999
2 , [4 4 3] --> [4]	 progress 438/3999
6 , [9 4 3] --> [3]	 progress 439/3999
2 , [0 2 2] --> [2]	 progress 440/3999
6 , [4 0 2] --> [0]	 progress 441/3999
8 , [7 4 7] --> [7]	 progress 442/3999
5 , [3 4 8] --> [3]	 progress 443/3999
4 , [0 5 0] --> [0]	 progress 444/3999
2 , [3 2 4] --> [2]	 progress 445/3999
9 , [9 8 5] --> [5]	 progress 446/3999
0 , [0 6 1] --> [0]	 progress 447/3999
3 , [9 8 4] --> [4]	 progress 448/3999
0 , [1 1 9] --> [1]	 progress 449/3999
6 , [8 4 1] --> [1]	 prog

1 , [2 0 4] --> [0]	 progress 637/3999
6 , [9 6 7] --> [6]	 progress 638/3999
4 , [4 2 0] --> [0]	 progress 639/3999
9 , [9 2 4] --> [2]	 progress 640/3999
9 , [7 8 1] --> [1]	 progress 641/3999
5 , [2 4 8] --> [2]	 progress 642/3999
2 , [4 3 5] --> [3]	 progress 643/3999
7 , [2 8 5] --> [2]	 progress 644/3999
3 , [9 4 7] --> [4]	 progress 645/3999
0 , [1 9 2] --> [1]	 progress 646/3999
4 , [4 1 3] --> [1]	 progress 647/3999
8 , [2 1 9] --> [1]	 progress 648/3999
8 , [6 9 6] --> [6]	 progress 649/3999
8 , [1 7 8] --> [1]	 progress 650/3999
7 , [2 2 8] --> [2]	 progress 651/3999
5 , [8 1 4] --> [1]	 progress 652/3999
7 , [5 3 4] --> [3]	 progress 653/3999
9 , [2 3 6] --> [2]	 progress 654/3999
3 , [2 3 9] --> [2]	 progress 655/3999
8 , [8 6 7] --> [6]	 progress 656/3999
5 , [4 0 4] --> [4]	 progress 657/3999
4 , [8 5 4] --> [4]	 progress 658/3999
2 , [4 9 4] --> [4]	 progress 659/3999
3 , [4 4 4] --> [4]	 progress 660/3999
2 , [6 9 8] --> [6]	 progress 661/3999
6 , [4 1 4] --> [4]	 prog

9 , [8 2 5] --> [2]	 progress 848/3999
1 , [4 1 7] --> [1]	 progress 849/3999
3 , [4 1 2] --> [1]	 progress 850/3999
5 , [5 1 5] --> [5]	 progress 851/3999
1 , [1 1 7] --> [1]	 progress 852/3999
7 , [2 0 7] --> [0]	 progress 853/3999
1 , [4 1 7] --> [1]	 progress 854/3999
8 , [4 1 5] --> [1]	 progress 855/3999
4 , [4 4 8] --> [4]	 progress 856/3999
4 , [1 2 0] --> [0]	 progress 857/3999
5 , [4 4 5] --> [4]	 progress 858/3999
8 , [2 8 6] --> [2]	 progress 859/3999
8 , [9 6 1] --> [1]	 progress 860/3999
6 , [4 2 4] --> [4]	 progress 861/3999
6 , [1 7 0] --> [0]	 progress 862/3999
3 , [1 3 7] --> [1]	 progress 863/3999
4 , [7 5 2] --> [2]	 progress 864/3999
0 , [8 5 9] --> [5]	 progress 865/3999
2 , [6 4 3] --> [3]	 progress 866/3999
7 , [3 2 5] --> [2]	 progress 867/3999
3 , [4 2 4] --> [4]	 progress 868/3999
7 , [7 9 4] --> [4]	 progress 869/3999
3 , [4 7 9] --> [4]	 progress 870/3999
3 , [6 8 2] --> [2]	 progress 871/3999
5 , [4 8 2] --> [2]	 progress 872/3999
6 , [8 3 6] --> [3]	 prog

...

9 , [4 3 0] --> [0]	 progress 3110/3999
4 , [7 0 2] --> [0]	 progress 3111/3999
3 , [2 4 7] --> [2]	 progress 3112/3999
9 , [8 3 1] --> [1]	 progress 3113/3999
0 , [1 1 4] --> [1]	 progress 3114/3999
0 , [9 8 4] --> [4]	 progress 3115/3999
3 , [8 0 8] --> [8]	 progress 3116/3999
4 , [4 4 6] --> [4]	 progress 3117/3999
7 , [5 1 9] --> [1]	 progress 3118/3999
9 , [2 0 0] --> [0]	 progress 3119/3999
7 , [3 2 4] --> [2]	 progress 3120/3999
9 , [7 6 5] --> [5]	 progress 3121/3999
8 , [0 4 2] --> [0]	 progress 3122/3999
8 , [4 6 0] --> [0]	 progress 3123/3999
1 , [3 2 5] --> [2]	 progress 3124/3999
3 , [0 7 9] --> [0]	 progress 3125/3999
0 , [3 0 9] --> [0]	 progress 3126/3999
6 , [6 6 6] --> [6]	 progress 3127/3999
8 , [4 8 1] --> [1]	 progress 3128/3999
2 , [7 1 4] --> [1]	 progress 3129/3999
8 , [4 1 3] --> [1]	 progress 3130/3999
8 , [7 4 1] --> [1]	 progress 3131/3999
8 , [3 9 6] --> [3]	 progress 3132/3999
4 , [9 1 6] --> [1]	 progress 3133/3999
6 , [0 6 9] --> [0]	 progress 3134/3999


5 , [2 6 4] --> [2]	 progress 3315/3999
0 , [7 9 1] --> [1]	 progress 3316/3999
0 , [2 4 4] --> [4]	 progress 3317/3999
5 , [1 1 9] --> [1]	 progress 3318/3999
5 , [5 6 8] --> [5]	 progress 3319/3999
7 , [1 9 0] --> [0]	 progress 3320/3999
6 , [2 8 2] --> [2]	 progress 3321/3999
9 , [5 8 2] --> [2]	 progress 3322/3999
1 , [0 9 7] --> [0]	 progress 3323/3999
7 , [4 4 4] --> [4]	 progress 3324/3999
7 , [4 6 3] --> [3]	 progress 3325/3999
3 , [4 4 3] --> [4]	 progress 3326/3999
2 , [2 2 6] --> [2]	 progress 3327/3999
0 , [5 7 1] --> [1]	 progress 3328/3999
2 , [2 1 3] --> [1]	 progress 3329/3999
4 , [6 9 6] --> [6]	 progress 3330/3999
0 , [4 8 7] --> [4]	 progress 3331/3999
9 , [2 6 8] --> [2]	 progress 3332/3999
5 , [2 8 1] --> [1]	 progress 3333/3999
5 , [0 7 7] --> [7]	 progress 3334/3999
8 , [0 2 1] --> [0]	 progress 3335/3999
2 , [4 8 7] --> [4]	 progress 3336/3999
5 , [1 0 6] --> [0]	 progress 3337/3999
0 , [1 8 4] --> [1]	 progress 3338/3999
4 , [3 8 3] --> [3]	 progress 3339/3999


4 , [0 2 5] --> [0]	 progress 3520/3999
1 , [6 4 7] --> [4]	 progress 3521/3999
9 , [9 0 5] --> [0]	 progress 3522/3999
5 , [4 3 5] --> [3]	 progress 3523/3999
9 , [5 7 2] --> [2]	 progress 3524/3999
1 , [7 1 2] --> [1]	 progress 3525/3999
0 , [2 9 2] --> [2]	 progress 3526/3999
4 , [4 1 5] --> [1]	 progress 3527/3999
0 , [0 1 8] --> [0]	 progress 3528/3999
8 , [1 8 2] --> [1]	 progress 3529/3999
8 , [2 5 7] --> [2]	 progress 3530/3999
4 , [0 7 9] --> [0]	 progress 3531/3999
1 , [4 0 2] --> [0]	 progress 3532/3999
9 , [4 2 4] --> [4]	 progress 3533/3999
7 , [4 7 0] --> [0]	 progress 3534/3999
2 , [0 7 8] --> [0]	 progress 3535/3999
5 , [8 4 6] --> [4]	 progress 3536/3999
9 , [5 4 4] --> [4]	 progress 3537/3999
1 , [8 6 9] --> [6]	 progress 3538/3999
5 , [6 6 2] --> [6]	 progress 3539/3999
0 , [4 8 5] --> [4]	 progress 3540/3999
7 , [0 1 9] --> [0]	 progress 3541/3999
4 , [8 5 2] --> [2]	 progress 3542/3999
3 , [6 6 2] --> [6]	 progress 3543/3999
6 , [2 4 7] --> [2]	 progress 3544/3999


3 , [2 4 2] --> [2]	 progress 3725/3999
3 , [9 5 0] --> [0]	 progress 3726/3999
0 , [4 5 8] --> [4]	 progress 3727/3999
2 , [6 2 4] --> [2]	 progress 3728/3999
0 , [8 2 6] --> [2]	 progress 3729/3999
6 , [5 8 1] --> [1]	 progress 3730/3999
6 , [9 7 8] --> [7]	 progress 3731/3999
8 , [8 4 5] --> [4]	 progress 3732/3999
2 , [2 2 2] --> [2]	 progress 3733/3999
5 , [2 7 0] --> [0]	 progress 3734/3999
1 , [2 6 8] --> [2]	 progress 3735/3999
2 , [3 4 0] --> [0]	 progress 3736/3999
3 , [5 2 8] --> [2]	 progress 3737/3999
7 , [8 2 8] --> [8]	 progress 3738/3999
0 , [4 4 4] --> [4]	 progress 3739/3999
0 , [1 6 6] --> [6]	 progress 3740/3999
4 , [7 4 1] --> [1]	 progress 3741/3999
2 , [4 4 8] --> [4]	 progress 3742/3999
7 , [2 7 6] --> [2]	 progress 3743/3999
1 , [6 8 8] --> [8]	 progress 3744/3999
4 , [7 8 0] --> [0]	 progress 3745/3999
7 , [1 2 7] --> [1]	 progress 3746/3999
8 , [1 1 8] --> [1]	 progress 3747/3999
0 , [3 3 2] --> [3]	 progress 3748/3999
3 , [0 2 2] --> [2]	 progress 3749/3999


7 , [5 8 1] --> [1]	 progress 3930/3999
2 , [1 8 9] --> [1]	 progress 3931/3999
5 , [4 9 9] --> [9]	 progress 3932/3999
9 , [0 3 5] --> [0]	 progress 3933/3999
1 , [2 6 6] --> [6]	 progress 3934/3999
4 , [2 5 3] --> [2]	 progress 3935/3999
8 , [0 4 6] --> [0]	 progress 3936/3999
5 , [4 1 1] --> [1]	 progress 3937/3999
1 , [1 1 5] --> [1]	 progress 3938/3999
7 , [7 6 9] --> [6]	 progress 3939/3999
6 , [8 6 1] --> [1]	 progress 3940/3999
0 , [2 6 8] --> [2]	 progress 3941/3999
4 , [8 2 6] --> [2]	 progress 3942/3999
2 , [1 2 6] --> [1]	 progress 3943/3999
0 , [6 1 9] --> [1]	 progress 3944/3999
1 , [2 0 7] --> [0]	 progress 3945/3999
1 , [4 4 8] --> [4]	 progress 3946/3999
8 , [8 1 2] --> [1]	 progress 3947/3999
6 , [2 2 1] --> [2]	 progress 3948/3999
2 , [6 1 9] --> [1]	 progress 3949/3999
7 , [4 7 6] --> [4]	 progress 3950/3999
4 , [8 6 4] --> [4]	 progress 3951/3999
9 , [1 1 6] --> [1]	 progress 3952/3999
8 , [2 1 3] --> [1]	 progress 3953/3999
6 , [4 4 3] --> [4]	 progress 3954/3999


In [ ]:
# X uint8
# knn gray with k=3, 20,000[16,000, 4000],  dtype=uint8  = 10.578%
# knn gray with k=3, 30,000[24,000, 6000],  dtype=uint8  = 10.502% (1827.51s->30.45m)

# X float64
# knn gray with k=3, 10,000[ 8,000, 2000], dtype=float64 = 11.056% (214.57s-> 3.57m)
# knn gray with k=3, 20,000[16,000, 4000], dtype=float64 = 10.578% (815.93s->13.59m)

In [ ]:
k= 1
test_sample = test_set[0]
l1_dict_vec = l1_dist(test_sample,train_set)
idx = np.argpartition(l1_dict_vec,k)
#
print(Y[train_set_num+0])
print(Y[train_set_num+idx[0]])

In [ ]:
int_v = test_set[0]/255
trn_v = train_set[0]/255
print(int_v)
print(trn_v)

In [ ]:
print('np.linalg.norm = ' + str(np.linalg.norm((int_v - trn_v), ord=2)))
dist_l1 = sum(abs(int_v - trn_v))
print('sum(abs(a-b)) = ' + str(dist_l1))


In [ ]:
int_v = np.array([1,1,1,1,1])
tring_set = np.array([[2,2,2,2,2],[3,3,3,3,3],[4,4,4,4,4],[5,5,5,5,5]])
#dist_lambda = lambda x: np.linalg.norm((int_v - x), ord=1)
dist_lambda = lambda x: sum(abs(int_v - x))
#dist_vec = map{lamda x: np.linalg.norm((int_v - x), ord=1),train_set}
dist_vec = dist_lambda(tring_set)
dist_vec